## Configuration

In [0]:
### google drive mount ###
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
### configuration ###
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime
from datetime import datetime, date

# for collecting open api data
import urllib.request
import json
from pandas.io.json import json_normalize

# config
DATA_PATH = "/content/gdrive/My Drive/data/"

In [0]:
### collect & concat data function ###

def make_path(add_path):
    path_list = [DATA_PATH]
    path_list.append(add_path)
    return ''.join(path_list)

def read_file(path, encode='CP949'):
    df = pd.read_csv(path, encoding=encode)
    return df

def read_files(path, encode='CP949'):
    df_from_each_file = (
        pd.read_csv(f, encoding=encode, dtype=str) for f in path
        )
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    return concatenated_df

## API Data Collect - 1) 역 주소




In [0]:
# 역코드
stn_cd_file_path = make_path("서울시_지하철역_코드.csv")
stn_cd_df = pd.read_csv(stn_cd_file_path, encoding='CP949', dtype=str)
stn_cd_df = stn_cd_df.loc[stn_cd_df['호선'].str.contains('호선')] #경의,인천선 잘라내기
stn_cd_df['호선']= stn_cd_df.호선.str.split('0').str[1].str.strip() #호선 번호만 남기기
stn_cd_df.rename(columns=                  
             {'전철역명': '지하철역',
              '호선':'호선명'},                   
             inplace=True)
stn_cd_df['호선명'] = stn_cd_df.호선명.str.split('호선').str[0].str.strip()
stn_cd_df['호선명'] = stn_cd_df['호선명'].fillna(-1).astype('int')
stn_cd_df = stn_cd_df[(stn_cd_df.호선명 > 0) & (stn_cd_df.호선명 <5)] # 1~4호선
stn_cd_df.head(3)

,전철역코드,지하철역,호선명,외부코드
0,0333,매봉,3,343
1,0334,도곡,3,344
2,0335,대치,3,345


In [0]:
# api 호출할 역코드 list
str_list = stn_cd_df['전철역코드'].apply(str).tolist()
stn_cd_list = list(set(str_list))
len(stn_cd_list)

241

In [0]:
# API Call 
key = "62426f765573736f3830475a724141"

df = pd.DataFrame()

for stn_cd in stn_cd_list:
    url = "http://openapi.seoul.go.kr:8088/%s/json/SearchSTNInfoByIDService/1/1/%s/" %(key, stn_cd)
    response = urllib.request.urlopen(url)
    print ('Your Request:\n'+url)
    response = urllib.request.urlopen(url)
    json_str = response.read().decode("utf-8")
    json_object = json.loads(json_str)
    if stn_cd != '0200': # 까치산역 제외
        api_data = pd.json_normalize(json_object['SearchSTNInfoByIDService']['row'])
        df = df.append(api_data)
        print('Append Success :\nstation cd %s %d rows' % (stn_cd, len(api_data)))

Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/0150/
Append Success :
station cd 0150 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/1762/
Append Success :
station cd 1762 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/0312/
Append Success :
station cd 0312 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/0339/
Append Success :
station cd 0339 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/1803/
Append Success :
station cd 1803 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573736f3830475a724141/json/SearchSTNInfoByIDService/1/1/0244/
Append Success :
station cd 0244 1 rows
Your Request:
http://openapi.seoul.go.kr:8088/62426f765573

,STATION_CD,STATION_NM,STATION_NM_ENG,LINE_NUM,FR_CODE,CYBER_ST_CODE,BRANCH_OFFICE_CODE,BRANCH_OFFICE_NM,REGIDATE,TEL,FAX,POST_CODE,ADDRESS,MINWON,INFOTESK,CULTURE,TICKET,OBSTACLE,MEETPLACE,PARKING,BICYCLE,MUIN,OFFICE,NURSING,ELEVATOR,ESCALATOR,WHEELCHAIR,SPEEDNATE,TOILET,STATION_NM_HAN,URE,XPOINT,YPOINT,XPOINT_WGS,YPOINT_WGS
0,0150,서울,Seoul,1,133,0150,115100,시청영업소,1985-10-18 0:0:0,02-6110-1331,6110 - 1339,04527,서울특별시 중구 세종대로 지하 2,N,N,N,Y,Y,Y,N,Y,N,Y,N,Y,Y,Y,Y,게이트 외부,서울,,493365.0,1125595.0,37.554648,126.972559
0,1762,오이도,Oido,4,456,1762,410000,철도청구간,,031-319-7788,,15018,경기도 시흥시 역전로 430,,,,,,,,,,,N,,,,,게이트 외부,烏耳島,,441955.0,1072345.0,37.362357,126.738714
0,0312,불광,Bulgwang,3,322,0312,131700,경복궁영업소,1985-7-12 0:0:0,02-6110-3221,6110 - 3229,03397,서울특별시 은평구 통일로 지하 723-1,N,N,N,N,Y,Y,N,Y,N,Y,N,Y,Y,N,N,게이트 외부,佛光,,484337.0,1140982.0,37.610469,126.929887
0,0339,수서,Suseo,3,349,0339,133400,도곡영업소,1993-10-30 0:0:0,02-6110-3491,6110 - 3499,06362,서울특별시 강남구 광평로 지하 270,Y,N,N,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,게이트 내부,水西,,522445.0,1106977.0,37.487371,127.101880
0,1803,역곡,Yeokgok,1,146,1803,410000,철도청구간,,02-2639-3434,,14673,경기도 부천시 원미구 역곡로 1,,,,,,,,,,,N,,,,,게이트 외부,驛谷,조선시대에는 40리마다 역원이 설치되어 있었으며 한양과 제물포사이 40리 지점에도 ...,458312.0,1106430.0,37.485178,126.811502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0417,길음,Gireum,4,417,0417,141700,상계서비스센터,1985-04-20 00:00:00,02-6110-4171,6110 - 4179,02735,서울특별시 성북구 동소문로 지하 248,N,N,N,N,N,Y,N,Y,N,Y,Y,Y,N,N,Y,게이트 외부,吉音,○ 길음동은 정릉천의 골짜기가 길게 놓여 있어 기리묵골이라 하던 것을 한자음으로 고...,505320.0,1139202.0,37.603407,127.025053
0,1021,녹천,Nokcheon,1,117,1021,410000,철도청구간,,02-905-0182,,01488,서울특별시 도봉구 덕릉로 376,,,,,,,,,,,N,,,,,게이트 내부,鹿川,이 지역에 홍수로 인해 폐허가 된 마을 뒷산에서 사슴이 내려와 마을앞 중랑천에서 목...,511557.0,1150497.0,37.644799,127.051269
0,0313,녹번,Nokbeon,3,323,0313,131700,경복궁영업소,1985-07-12 00:00:00,02-6110-3231,6110 - 3239,03373,서울특별시 은평구 통일로 지하 602-1,N,N,N,N,N,Y,N,N,Y,Y,N,Y,N,N,Y,게이트 외부,碌磻,○ 녹번이란 지명은 '녹번이' 란 고개에서 녹색을 띠는 천연구리(한방약용)가 나와 ...,485672.0,1138467.0,37.600927,126.935756
0,1705,관악,Gwanak,1,P146,1705,410000,철도청구간,,031-471-7788,,13960,경기도 안양시 만안구 경수대로 1273번길 46,,,,,,,,,,,N,,,,,"게이트 내부,외부",冠岳,주변에 관악산이 있어 대표적인 명칭으로 명명.,479698.0,1088013.0,37.419232,126.908706


In [0]:
# 역 주소 정보 출력 
df

In [0]:
# 수집한 역주소 csv로 저장 (두번 이상 호출하는 일 없게)
raw_addr_path = make_path("addr.csv")
df.to_csv(raw_addr_path, encoding="CP949")

## API Data Collect - 2) 지하철 시간표

In [0]:
# API Call 

key = "416653794d73736f343173776b6c72"
inout_tag_list = ["1","2"] # 상행,내선:1 하행,외선:2
week_tag = "1" # 평일:1

df = pd.DataFrame()

for stn_cd in stn_cd_list:
    for inout_tag in inout_tag_list:
        url = "http://openapi.seoul.go.kr:8088/%s/json/SearchSTNTimeTableByIDService/1/1000/%s/%s/%s/" %(key, stn_cd, week_tag, inout_tag)
        response = urllib.request.urlopen(url)
        json_str = response.read().decode("utf-8")
        json_object = json.loads(json_str)
        api_data = pd.json_normalize(json_object['SearchSTNTimeTableByIDService']['row'])
        df = df.append(api_data)
        print('Append Success :\nstation cd %s %d rows' % (stn_cd, len(api_data)))

Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/0150/1/1/
Append Success :
station cd 0150 264 rows
Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/0150/1/2/
Append Success :
station cd 0150 259 rows
Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/1762/1/1/
Append Success :
station cd 1762 108 rows
Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/1762/1/2/
Append Success :
station cd 1762 111 rows
Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/0312/1/1/
Append Success :
station cd 0312 187 rows
Your Request:
http://openapi.seoul.go.kr:8088/416653794d73736f343173776b6c72/json/SearchSTNTimeTableByIDService/1/1000/0312/1/2/
Append Success :

In [0]:
# 시간표 정보 출력
df

,LINE_NUM,FR_CODE,STATION_CD,STATION_NM,TRAIN_NO,ARRIVETIME,LEFTTIME,ORIGINSTATION,DESTSTATION,SUBWAYSNAME,SUBWAYENAME,WEEK_TAG,INOUT_TAG,FL_FLAG,DESTSTATION2,EXPRESS_YN,BRANCH_LINE
0,01호선,133,0150,서울역,K1902,07:17:00,00:00:00,1728,0150,천안,서울역,1,1,,,D,
1,01호선,133,0150,서울역,S222,23:58:30,00:00:00,1812,0150,인천,서울역,1,1,,,G,
2,01호선,133,0150,서울역,K1904,08:06:00,00:00:00,1408,0150,신창,서울역,1,1,,,D,
3,01호선,133,0150,서울역,K802,05:19:30,05:20:00,1701,1915,구로,동두천,1,1,,,G,
4,01호선,133,0150,서울역,S902,00:00:00,05:24:00,0150,1906,서울역,의정부,1,1,,,G,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,04호선,430,0430,이촌,4727K,23:14:30,23:15:00,0409,1751,당고개,산본,1,2,,,G,
224,04호선,430,0430,이촌,4205,23:22:30,23:23:00,0409,0433,당고개,사당,1,2,,,G,
225,04호선,430,0430,이촌,4729K,23:31:30,23:32:00,0409,1762,당고개,오이도,1,2,,,G,
226,04호선,430,0430,이촌,4207,23:41:30,23:42:00,0409,0433,당고개,사당,1,2,,,G,


In [0]:
# 수집한 시간표 csv로 저장 (두번 이상 호출하는 일 없게)
raw_timechart_path = make_path("timechart.csv")
df.to_csv(raw_timechart_path)

## API Data Collect - 3) SKT 유동인구

In [0]:
skt_access_key = "f5c2f25010e43c38792973bc8d48816fae7c1ef8394a23edd5899cf9071ce486"
df = pd.DataFrame()

for i in range(1,11):
    url = "https://api.bigdatahub.co.kr/v1/datahub/datasets/search.json?pid=1002261&TDCAccessKey=%s&$page=%d&$count=2157" %(skt_access_key, i)
    response = urllib.request.urlopen(url)
    json_str = response.read().decode("utf-8")
    json_object = json.loads(json_str)
    api_data = pd.json_normalize(json_object['entry'])
    df = df.append(api_data)
df.head(10)